In [3]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#from scipy.misc import imread
#from wordcloud import WordCloud, STOPWORDS
from nltk.corpus import stopwords 

import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.offline as py
import plotly.graph_objs as go
import plotly.tools as tls
from bs4 import BeautifulSoup

import string
from string import digits
from nltk.tokenize import word_tokenize, TreebankWordTokenizer
from nltk import SnowballStemmer, PorterStemmer, LancasterStemmer
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder


from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import learning_curve
from sklearn.model_selection import train_test_split
from sklearn import metrics

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB


class color:
     BOLD = '\033[1m'
     UNDERLINE = '\033[4m'
     END = '\033[0m'

In [5]:
solution = pd.read_csv('sample_submission.csv')

train = pd.read_csv('train.csv')

test = pd.read_csv('test.csv')

In [6]:
solution.head()

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0


In [7]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [8]:
test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [10]:
solution.shape,train.shape,test.shape

((3263, 2), (7613, 5), (3263, 4))

In [11]:
train.isnull().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [12]:
test.isnull().sum()

id             0
keyword       26
location    1105
text           0
dtype: int64

In [13]:
from nltk.stem import WordNetLemmatizer
stemmer = SnowballStemmer('english')
lemma = WordNetLemmatizer()

In [14]:
test['target'] = None

In [15]:
df = pd.concat([train,test])

In [16]:
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [17]:
df.shape

(10876, 5)

In [18]:
df1 = df.drop(['id', 'keyword', 'location'], axis = 1)
df1.head()

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


In [76]:
def text_process(mess):
    
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
   # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]

    #Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    #Now just remove any stopwords
    no_stop = [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]
    
    return no_stop#[stemmer.stem(w) for w in mess] #[lemma.lemmatize(w) for w in mess]

In [82]:
Vect = CountVectorizer(tokenizer = text_process, ngram_range=(1, 3), stop_words='english')#,max_features = 4000)#, max_features = 5000)ngram_range=(1, 3)

In [83]:
X = train['text'].astype(str)

y = train['target'].astype(str)

In [84]:
# K_folds:
from sklearn.model_selection import KFold

kf = KFold(n_splits = 5, shuffle = True)

from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.multiclass import OneVsRestClassifier

In [23]:
X = Vect.fit_transform(X)

In [37]:
# from imblearn.pipeline import make_pipeline as mp
# from imblearn.over_sampling import SMOTE
# from imblearn.pipeline import make_pipeline
# from imblearn.metrics import classification_report_imbalanced
# from collections import Counter
# from sklearn import metrics
# from sklearn.ensemble import GradientBoostingClassifier
# from imblearn.under_sampling import RandomUnderSampler
# from imblearn.over_sampling import RandomOverSampler
# from imblearn.over_sampling import SMOTENC, ADASYN

In [26]:
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.multiclass import OneVsRestClassifier

In [85]:
X_train, X_test, y_train, y_test= train_test_split(X , y, test_size=0.2,random_state = 0)

In [111]:
from catboost import CatBoostClassifier

In [112]:
LRmodel = OneVsRestClassifier(CatBoostClassifier())##LogisticRegression(C=100, dual=True, random_state=0 )
pipe = Pipeline([('vec', Vect), ('model', LRmodel)])

In [35]:
# scores = []
# for i in range (5):
#     result = next(kf.split(X), None)
#     X_train = X.iloc[result[0]]
#     X_test = X.iloc[result[1]]
#     y_train = y.iloc[result[0]]
#     y_test = y.iloc[result[1]]
#     model = pipe.fit(X_train, y_train)
#     scores.append(model.score(X_test,y_test))
# print(scores)
# print(np.mean(scores))

In [113]:
pipe.fit(X_train , y_train)

Learning rate set to 0.027936
0:	learn: 0.6912320	total: 4.83s	remaining: 1h 20m 22s
1:	learn: 0.6878268	total: 6.64s	remaining: 55m 15s
2:	learn: 0.6856970	total: 7.86s	remaining: 43m 33s
3:	learn: 0.6829961	total: 8.77s	remaining: 36m 24s
4:	learn: 0.6816009	total: 9.72s	remaining: 32m 14s
5:	learn: 0.6797293	total: 10.7s	remaining: 29m 31s
6:	learn: 0.6777375	total: 11.7s	remaining: 27m 42s
7:	learn: 0.6761307	total: 12.7s	remaining: 26m 18s
8:	learn: 0.6749104	total: 13.7s	remaining: 25m 4s
9:	learn: 0.6729334	total: 14.6s	remaining: 24m 5s
10:	learn: 0.6718197	total: 16.6s	remaining: 24m 55s
11:	learn: 0.6704783	total: 18.6s	remaining: 25m 28s
12:	learn: 0.6696826	total: 20.1s	remaining: 25m 25s
13:	learn: 0.6684349	total: 21.4s	remaining: 25m 3s
14:	learn: 0.6677333	total: 22.5s	remaining: 24m 39s
15:	learn: 0.6667558	total: 24.3s	remaining: 24m 57s
16:	learn: 0.6655619	total: 26.2s	remaining: 25m 15s
17:	learn: 0.6642834	total: 27.8s	remaining: 25m 14s
18:	learn: 0.6632342	total

153:	learn: 0.5924832	total: 2m 41s	remaining: 14m 45s
154:	learn: 0.5920253	total: 2m 42s	remaining: 14m 43s
155:	learn: 0.5917408	total: 2m 42s	remaining: 14m 41s
156:	learn: 0.5914253	total: 2m 43s	remaining: 14m 39s
157:	learn: 0.5912002	total: 2m 44s	remaining: 14m 37s
158:	learn: 0.5909022	total: 2m 45s	remaining: 14m 35s
159:	learn: 0.5905509	total: 2m 46s	remaining: 14m 33s
160:	learn: 0.5902535	total: 2m 47s	remaining: 14m 31s
161:	learn: 0.5899381	total: 2m 48s	remaining: 14m 29s
162:	learn: 0.5897372	total: 2m 48s	remaining: 14m 27s
163:	learn: 0.5894603	total: 2m 49s	remaining: 14m 25s
164:	learn: 0.5891221	total: 2m 50s	remaining: 14m 23s
165:	learn: 0.5889528	total: 2m 51s	remaining: 14m 21s
166:	learn: 0.5886096	total: 2m 52s	remaining: 14m 20s
167:	learn: 0.5884061	total: 2m 53s	remaining: 14m 18s
168:	learn: 0.5881512	total: 2m 54s	remaining: 14m 16s
169:	learn: 0.5878223	total: 2m 55s	remaining: 14m 15s
170:	learn: 0.5875933	total: 2m 56s	remaining: 14m 13s
171:	learn

303:	learn: 0.5531645	total: 4m 57s	remaining: 11m 20s
304:	learn: 0.5528326	total: 4m 57s	remaining: 11m 18s
305:	learn: 0.5525457	total: 4m 58s	remaining: 11m 17s
306:	learn: 0.5521446	total: 4m 59s	remaining: 11m 16s
307:	learn: 0.5517845	total: 5m	remaining: 11m 15s
308:	learn: 0.5515210	total: 5m 1s	remaining: 11m 13s
309:	learn: 0.5511902	total: 5m 2s	remaining: 11m 12s
310:	learn: 0.5509569	total: 5m 3s	remaining: 11m 11s
311:	learn: 0.5505468	total: 5m 3s	remaining: 11m 10s
312:	learn: 0.5502556	total: 5m 4s	remaining: 11m 9s
313:	learn: 0.5498065	total: 5m 5s	remaining: 11m 7s
314:	learn: 0.5495079	total: 5m 6s	remaining: 11m 7s
315:	learn: 0.5489713	total: 5m 7s	remaining: 11m 6s
316:	learn: 0.5486871	total: 5m 8s	remaining: 11m 5s
317:	learn: 0.5482342	total: 5m 9s	remaining: 11m 3s
318:	learn: 0.5478914	total: 5m 10s	remaining: 11m 2s
319:	learn: 0.5476962	total: 5m 11s	remaining: 11m 1s
320:	learn: 0.5473087	total: 5m 12s	remaining: 11m
321:	learn: 0.5470173	total: 5m 13s	

455:	learn: 0.5063714	total: 7m 17s	remaining: 8m 42s
456:	learn: 0.5061592	total: 7m 18s	remaining: 8m 41s
457:	learn: 0.5058772	total: 7m 19s	remaining: 8m 40s
458:	learn: 0.5056385	total: 7m 20s	remaining: 8m 39s
459:	learn: 0.5054324	total: 7m 21s	remaining: 8m 38s
460:	learn: 0.5052085	total: 7m 22s	remaining: 8m 37s
461:	learn: 0.5049641	total: 7m 23s	remaining: 8m 35s
462:	learn: 0.5046908	total: 7m 23s	remaining: 8m 34s
463:	learn: 0.5044545	total: 7m 24s	remaining: 8m 33s
464:	learn: 0.5042093	total: 7m 25s	remaining: 8m 32s
465:	learn: 0.5039551	total: 7m 26s	remaining: 8m 31s
466:	learn: 0.5036999	total: 7m 27s	remaining: 8m 30s
467:	learn: 0.5034955	total: 7m 28s	remaining: 8m 29s
468:	learn: 0.5032607	total: 7m 29s	remaining: 8m 28s
469:	learn: 0.5029614	total: 7m 30s	remaining: 8m 27s
470:	learn: 0.5027382	total: 7m 31s	remaining: 8m 26s
471:	learn: 0.5025551	total: 7m 32s	remaining: 8m 25s
472:	learn: 0.5023176	total: 7m 33s	remaining: 8m 24s
473:	learn: 0.5020691	total:

608:	learn: 0.4724551	total: 9m 41s	remaining: 6m 13s
609:	learn: 0.4722764	total: 9m 42s	remaining: 6m 12s
610:	learn: 0.4721282	total: 9m 43s	remaining: 6m 11s
611:	learn: 0.4719238	total: 9m 44s	remaining: 6m 10s
612:	learn: 0.4717459	total: 9m 45s	remaining: 6m 9s
613:	learn: 0.4715992	total: 9m 46s	remaining: 6m 9s
614:	learn: 0.4714182	total: 9m 47s	remaining: 6m 8s
615:	learn: 0.4712410	total: 9m 48s	remaining: 6m 7s
616:	learn: 0.4710317	total: 9m 49s	remaining: 6m 6s
617:	learn: 0.4707703	total: 9m 50s	remaining: 6m 5s
618:	learn: 0.4705116	total: 9m 51s	remaining: 6m 4s
619:	learn: 0.4703100	total: 9m 52s	remaining: 6m 3s
620:	learn: 0.4701070	total: 9m 53s	remaining: 6m 2s
621:	learn: 0.4699181	total: 9m 54s	remaining: 6m 1s
622:	learn: 0.4697303	total: 9m 55s	remaining: 6m
623:	learn: 0.4695845	total: 9m 56s	remaining: 5m 59s
624:	learn: 0.4694548	total: 9m 57s	remaining: 5m 58s
625:	learn: 0.4692558	total: 9m 58s	remaining: 5m 57s
626:	learn: 0.4690314	total: 9m 59s	remain

759:	learn: 0.4472380	total: 12m 6s	remaining: 3m 49s
760:	learn: 0.4470717	total: 12m 7s	remaining: 3m 48s
761:	learn: 0.4469963	total: 12m 8s	remaining: 3m 47s
762:	learn: 0.4468897	total: 12m 9s	remaining: 3m 46s
763:	learn: 0.4466924	total: 12m 10s	remaining: 3m 45s
764:	learn: 0.4465444	total: 12m 11s	remaining: 3m 44s
765:	learn: 0.4463791	total: 12m 11s	remaining: 3m 43s
766:	learn: 0.4462242	total: 12m 12s	remaining: 3m 42s
767:	learn: 0.4460805	total: 12m 13s	remaining: 3m 41s
768:	learn: 0.4459423	total: 12m 14s	remaining: 3m 40s
769:	learn: 0.4457568	total: 12m 15s	remaining: 3m 39s
770:	learn: 0.4456555	total: 12m 16s	remaining: 3m 38s
771:	learn: 0.4455187	total: 12m 17s	remaining: 3m 37s
772:	learn: 0.4453087	total: 12m 17s	remaining: 3m 36s
773:	learn: 0.4451641	total: 12m 18s	remaining: 3m 35s
774:	learn: 0.4450044	total: 12m 19s	remaining: 3m 34s
775:	learn: 0.4448472	total: 12m 20s	remaining: 3m 33s
776:	learn: 0.4446645	total: 12m 21s	remaining: 3m 32s
777:	learn: 0.

909:	learn: 0.4275889	total: 14m 29s	remaining: 1m 25s
910:	learn: 0.4273982	total: 14m 30s	remaining: 1m 25s
911:	learn: 0.4272565	total: 14m 31s	remaining: 1m 24s
912:	learn: 0.4271078	total: 14m 32s	remaining: 1m 23s
913:	learn: 0.4269674	total: 14m 33s	remaining: 1m 22s
914:	learn: 0.4268414	total: 14m 34s	remaining: 1m 21s
915:	learn: 0.4266555	total: 14m 35s	remaining: 1m 20s
916:	learn: 0.4264483	total: 14m 36s	remaining: 1m 19s
917:	learn: 0.4263868	total: 14m 37s	remaining: 1m 18s
918:	learn: 0.4263203	total: 14m 38s	remaining: 1m 17s
919:	learn: 0.4261593	total: 14m 39s	remaining: 1m 16s
920:	learn: 0.4260998	total: 14m 40s	remaining: 1m 15s
921:	learn: 0.4260213	total: 14m 41s	remaining: 1m 14s
922:	learn: 0.4258939	total: 14m 42s	remaining: 1m 13s
923:	learn: 0.4258288	total: 14m 43s	remaining: 1m 12s
924:	learn: 0.4257124	total: 14m 44s	remaining: 1m 11s
925:	learn: 0.4255839	total: 14m 44s	remaining: 1m 10s
926:	learn: 0.4254102	total: 14m 45s	remaining: 1m 9s
927:	learn:

Pipeline(memory=None,
         steps=[('vec',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 3), preprocessor=None,
                                 stop_words='english', strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function text_process at 0x000002290CE19EA0>,
                                 vocabulary=None)),
                ('model',
                 OneVsRestClassifier(estimator=<catboost.core.CatBoostClassifier object at 0x00000229116ADB38>,
                                     n_jobs=None))],
         verbose=False)

In [114]:
y_pred_test = pipe.predict(X_test)
y_pred_test

array(['0', '1', '1', ..., '1', '0', '0'], dtype='<U1')

In [115]:
print('Accuracy: '+ str(metrics.accuracy_score(y_test, y_pred_test)))
print('Precision: '+ str( metrics.precision_score(y_test, y_pred_test, average='weighted')))
print('Recall: '+ str(metrics.recall_score(y_test, y_pred_test, average='weighted')))
print('F1_Score: '+ str( metrics.f1_score(y_test, y_pred_test, average='weighted')))

Accuracy: 0.7852921864740644
Precision: 0.7959623276428103
Recall: 0.7852921864740644
F1_Score: 0.776938533457145


In [116]:
print(metrics.classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       0.76      0.92      0.83       886
           1       0.85      0.59      0.70       637

    accuracy                           0.79      1523
   macro avg       0.80      0.76      0.77      1523
weighted avg       0.80      0.79      0.78      1523



In [117]:
metrics.confusion_matrix(y_test, y_pred_test)

array([[817,  69],
       [258, 379]], dtype=int64)

In [105]:
y_pred_TK = pipe.predict(test['text'])

In [106]:
y_pred_TK

array(['0', '1', '1', ..., '1', '1', '1'], dtype='<U1')

In [107]:
test['target'] = y_pred_TK
test.head()

,id,keyword,location,text,target
0,0,NaN,NaN,Just happened a terrible car crash,0
1,2,NaN,NaN,"Heard about #earthquake is different cities, s...",1
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...",1
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,0
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,1


In [108]:
df_lm = pd.DataFrame({"id":test['id'], "target":test['target']})
df_lm.head()

,id,target
0,0,0
1,2,1
2,3,1
3,9,0
4,11,1


In [109]:
df_lm.shape

(3263, 2)

In [110]:
df_lm.to_csv('df_randomforest.csv', index = False)

In [ ]:
df_one_noremiove = 0.77811
df_logregNostop = 0.78323
df_NostopPunc = 0.79652